In [22]:
import pandas as pd
import re
from nltk.tokenize import sent_tokenize

In [27]:
df = pd.read_csv('resource/msia_jobs.csv')
df.head()

,Job_Title,Company,Salary,Country,Location,Date_Posted,Description,Job_URL,Keywords_Present,Title_Keywords
0,Equipment Engineer (RFID Die Attach),Avery Dennison,NaN,Malaysia,Kulim,2020-12-20,Company DescriptionAbout Avery DennisonAvery D...,https://www.indeed.com.my/rc/clk?jk=3214bd7467...,"'Golang', 'R', 'C'",NaN
1,IT MSD Automation Engineer - Graduate Trainee,Intel,NaN,Malaysia,Kulim,2020-12-20,Job DescriptionDiagnose and solve a range of i...,https://www.indeed.com.my/rc/clk?jk=adebb34d3a...,"'Golang', 'R', 'Excel', 'C'",'Graduate'
2,Senior Design Engineer,Protenga Sdn Bhd,"RM 4,000 - RM 7,000 a month",Malaysia,Senai,2020-12-20,"As the Design Engineer/Draftman, you will deve...",https://www.indeed.com.my/company/Protenga-Sdn...,"'R', 'AutoCAD', 'C'",'Senior'
3,Manufacturing Engineer,HP,NaN,Malaysia,Penang,2020-12-20,HP is the world’s leading personal systems and...,https://www.indeed.com.my/rc/clk?jk=bd2d96b3c9...,"'Golang', 'R', 'Excel', 'C'",NaN
4,Senior Mechatronics Engineer,Protenga Sdn Bhd,"RM 4,000 - RM 6,000 a month",Malaysia,Senai,2020-12-20,"As the Mechatronics Engineer, you are one of t...",https://www.indeed.com.my/company/Protenga-Sdn...,"'R', 'C'",'Senior'


In [28]:
len(df)

3963

In [29]:
df2 = df.drop_duplicates(subset='Description')
df2 = df2.sample(n=1000, random_state=42)
len(df2)

1000

In [63]:
def clean_sent_tokenize(sent):
    sentences = []
    for s in sent_tokenize(sent):
        slices = []
        for match in re.finditer('[a-zA-Z]\.[A-Z]', s):
            slices.append(match.start()+2)
        slices.append(len(s))
        offset = 0
        subsentences = []
        for pos in sorted(slices):
            subsent = s[offset:pos]
            offset += len(subsent)
            sentences.append(subsent)
    return sentences

def clean_sent_tokenize(sent, threshold=5):
    sent = re.split('[.!?]', sent)
    final = []
    for subsent in sent:
        # subsent = re.findall('[A-Z][a-z]*', subsent)
        # subsent = re.sub(r'([A-Z][a-z])', r'.\1', subsent).split('.')
        subsent = re.sub(r'(?<=[a-z])(?=[A-Z])', r'.', subsent).split('.')
        for s in subsent:
            if len(s) >= threshold:
                final.append(s.strip())
    return final

In [156]:
s = 'WHY JOIN US? Hello.This is stupid. Please!!! Example i.e. this that. No wayPlease. Back-end Development and Product'
sent_tokenize(s)

['WHY JOIN US?',
 'Hello.This is stupid.',
 'Please!!!',
 'Example i.e.',
 'this that.',
 'No wayPlease.',
 'Back-end Development and Product']

In [157]:
clean_sent_tokenize(s)

['WHY JOIN US',
 'Hello',
 'This is stupid',
 'Please',
 'Example i',
 'this that',
 'No way',
 'Please',
 'Back-end Development and Product']

In [172]:
dict_list = []

for i, row in df2.iterrows():
    sent = re.split('[.!?;]', row['Description'])
    for j, s in enumerate(sent):
        if len(s) >= 5 and len(s.split(' ')) >= 2:
            dict_list.append({'Id': i, 'SentNum': j, 'Sent': s.strip()})

df_s = pd.DataFrame.from_dict(dict_list)
df_s.head()

,Id,SentNum,Sent
0,3941,0,WHY JOIN US
1,3941,1,We practise a vibrant & energetic office culture
2,3941,2,We provide opportunities for career advancemen...
3,3941,3,Good performance is always rewarded accordingly
4,3941,4,JOB RESPONSIBILITIES:What you will do:Communic...


In [173]:
len(df_s)

20496

In [174]:
df_s.to_csv('job_desc_sent.csv', index=False)